In [1]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from test_csmc import default_potential, nondefault_potential, test_gmm_ensemble_SIS

In [2]:
from csmc import ULAEnsembleCSMC

To Do list:
1. write a `test_mahalanobis` test
2. test brownian motion and plot
3. test AIS (with a MALA proposal)
4. test ULA
5. write and test uncontrolled CSMC
    - write function
    - assert uncontrolled ULA yields ULA logw with zero twisting potentials
6. create a twisted smc class that allows forr custom pytorch module inputs
    -test on uncontrolled smc
7. write and test a second (and generalizable) twist that adds twists
8. test on a 2d test problem
9. test on a 3N problem with internal coordinates

## 1. `test_mahalonobis`
done

## 2. `test_brownian motion`
done

## 3. (and 4.) test AIS (with a MALA proposal) and SIS (ULA)

layout of this AIS proposal...
1. allow for option to metropolize
2. if not metropolized, then we must be able to tally the forward/backward work proposal (a.k.a. shadow work in stat mech literature)
3. write a test system in 1d that anneals between two gaussians (with the same $\mu$ and increases $\Sigma$)
4. add a test for the wrapped ULA logw calculator.

### DONE!

## 5. write and test uncontrolled CSMC

it will have to look like this...
### DONE

In [4]:
test_gmm_ensemble_SIS()  

100%|██████████| 1000/1000 [00:05<00:00, 167.48it/s]

AIS : F = -0.339291310449628; dF = 0.009357865531732245
SIS : F = -0.3608197492547074; dF = 0.012665645977340895
uncontrolled smc: F = -0.34022332711807834; dF = 0.010718901687314363


so since we are recovering the correct free energy for a simple system, i am inclined to believe that the uncontrolled SMC is stable and working.

## 6. create a twisted smc class that allows forr custom pytorch module inputs

for every iteration of approximate dynamic programming, i need to input an existing twisting function $\psi$, $N \in \mathbf{N}$ i.i.d. samples from $Q^{\psi}$ and do the following...
1. set the log_k_twisted normalizer to 0.
2. iterate backward in time from T to 1 and:
    - set V_t_flat to -log_twisted_weight

ADP for learning optimal sequences boils down to having a method that can perform MSE regression to find appropriate twisting parameters.

attempt to run uncontrolled smc

fuck yeah! we passed the uncontrolled test.

### 6.1 write some twisting tests
In order to do this, I have to create some simple twisting sequences. for the purpose of this test, it will be a `nn.Sequential` with only a single `nn.Linear` layer

In [3]:
import torch.nn as nn
import torch

In [4]:
class Naught(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, x=None):
        return torch.zeros(1)
class MatrixReshape(nn.Module):
    def __init__(self, final_dim):
        super().__init__()
        self.final_dim = final_dim
    def forward(self, x=None):
        batch_size = x.size()[0]
        return x.reshape(batch_size, self.final_dim, self.final_dim)

In [5]:
A0 = nn.Sequential(Naught(), nn.Linear(1,1), MatrixReshape(1))
b0 = nn.Sequential(Naught(), nn.Linear(1,1))
c0 = nn.Sequential(Naught(), nn.Linear(1,1))

In [6]:
A = nn.Sequential(nn.Linear(1,1), MatrixReshape(1))
b = nn.Sequential(nn.Linear(1,1))
c = nn.Sequential(Naught(), nn.Linear(1,1))

In [7]:
input = torch.tensor([[[1.]], [[2.]], [[3.]]])

In [8]:
input.size()

torch.Size([3, 1, 1])

In [9]:
A(input)

tensor([[[ 0.2856]],

        [[ 0.1059]],

        [[-0.0738]]], grad_fn=<ViewBackward>)

In [10]:
csmc = ULAEnsembleCSMC(potential=nondefault_potential,
                      parameter_sequence=np.linspace(0,1,10),
                      dt=1.,
                      gmm_mixing_components=np.array([1.]),
                      gmm_means = np.array([[0.]]),
                      gmm_covariance_matrices=np.array([[[1.]]]),
                      A0_twisting_model=A0,
                      b0_twisting_model=b0,
                      c0_twisting_model=c0,
                      A_twisting_model=A,
                      b_twisting_model=b,
                      c_twisting_model=c)

In [11]:
logws, traj = csmc.run_twisted_smc(20)

100%|██████████| 20/20 [00:00<00:00, 158.00it/s]


In [12]:
csmc.ADP(
            twisted_logws = logws,
            xs = traj,
            train_batch_size = 10,
            train_epochs = 100,
            compute_optimal_ds=False,
            validation_fraction = None,
            validation_batch_size = None,
            optimizer = torch.optim.SGD,
            optimizer_kwargs = {'lr': 1e-3}
           )

100%|██████████| 10/10 [00:01<00:00,  5.82it/s]


In [13]:
a = torch.randn(23)

In [11]:
len(a)

23

In [3]:
from torch_utils import render_dataloaders
import torch

In [4]:
render_dataloaders(torch.from_numpy(np.random.randn(100)),torch.from_numpy(np.random.randn(100)), 0.2, 10, 20)

(<torch.utils.data.dataloader.DataLoader at 0x7f027da0da58>,
 <torch.utils.data.dataloader.DataLoader at 0x7f027da0db00>)

In [5]:
a = (1,2,3)

In [6]:
one, two, three = a

In [7]:
a = torch.tensor([[1., 2.], [3.,4.]])

In [8]:
a.numpy()

array([[1., 2.],
       [3., 4.]], dtype=float32)

In [38]:
a.repeat(10)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [35]:
a = np.random.randn(3,4).astype('float32')

In [36]:
q = torch.from_numpy(a)

In [37]:
q.type()

'torch.FloatTensor'

In [11]:
from torch_utils import batch_quadratic

In [12]:
batch_quadratic(torch.randn(10,1), torch.randn(10, 1, 1)).squeeze()

vshape, Mshape: [10, 1], [10, 1, 1]


tensor([-0.0243, -0.4487, -2.0009, -3.1136,  0.5495,  0.0118,  0.7648,  0.1788,
        -0.0768,  0.2280])

In [13]:
x = torch.randn(10, 1)
y = torch.randn(10, 2)
z = torch.randn(10)

In [14]:
x_kwarg

NameError: name 'x_kwarg' is not defined

In [15]:
torch.cat((x, y, z), 1)

RuntimeError: invalid argument 0: Tensors must have same number of dimensions: got 2 and 1 at /opt/conda/conda-bld/pytorch_1579022034529/work/aten/src/TH/generic/THTensor.cpp:603

In [16]:
for key, val in {}.items():
    print(yes)

In [17]:
any(item[0] == 1 for item in [])

False

In [30]:
def test_batch_dot():
    """
    test the batch mahalanobis against numpy
    """
    from scipy.spatial.distance import mahalanobis
    from torch_utils import batch_dot
    torch_v = torch.randn(10,3)
    torch_u = torch.randn(10, 3)
    batch_out = batch_dot(torch_v, torch_u)
    
    np_outs = []
    for v, u in zip(torch_v, torch_u):
        np_v, np_u = v.numpy(), u.numpy()
        np_outs.append(np_v.dot(np_u))
    assert np.allclose(np.array(np_outs), batch_out.numpy()), f"failed: {np_outs}, {batch_out}"

In [31]:
test_batch_dot()

In [16]:
from torch.nn.functional import mse_loss



In [19]:
torch.nn.functional.mse_loss(torch.randn(10), torch.randn(10))

tensor(1.4550)